In [ ]:
!pip install numpy
!pip install pandas 
!pip install matplotlib

# Frequências Características de falha
**Frequência devido à falha local na pista externa ($f_e$):**
$$
f_e = \frac{n f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha local na pista interna ($f_i$):**
$$
f_i = \frac{n f_r}{2} \left( 1 + \frac{d}{D} \cos \phi \right)
$$

**Frequência fundamental da carcaça ($f_C$):**
$$
f_C = \frac{f_r}{2} \left( 1 - \frac{d}{D} \cos \phi \right)
$$

**Frequência devido à falha na esfera ($f_B$):**
$$
f_B = \frac{D f_r}{2d} \left[ 1 - \left( \frac{d}{D} \cos \phi \right)^2 \right]
$$

# Amplitudes de Falha em função do diâmetro da falha

#### Falha na Esfera

1730 RPM Esfera (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :------------- | :-------------- | :----------------- |
| 0.007"         | 0.000076        | 68.906512          |
| 0.014"         | 0.000052        | 67.007271          |
| 0.021"         | 0.000061        | 67.203773          |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000052 | 67.007271 |
| 1750 | 0.000048 | 66.988548 |
| 1772 | 0.000038 | 70.937316 |
| 1797 | 0.000027 | 71.894030 |


#### Falha na Pista Interna

Falha na Pista Interna a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.005900 | 154.933817 |
| 0.014" | 0.008288 | 155.495846 |
| 0.021" | 0.011044 | 155.421302 |


Diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.008288 | 155.495846 |
| 1750 | 0.009055 | 157.674442 |
| 1772 | 0.009426 | 159.644141 |
| 1797 | 0.007026 | 161.712325 |

#### Pista Externa

Falha na Pista Externa a 1730 RPM para diâmetros diferentes (Sensor Drive End)

| diametro_falha | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 0.007" | 0.002440 | 103.384977 |
| 0.014" | 0.000360 | 103.286308 |
| 0.021" | 0.002292 | 102.991204 |


Falha na Pista Externa para diâmetro de falha de 0.014" (Sensor Drive End)

| rpm | amplitude_falha | freq_encontrada_hz |
| :--- | :--- | :--- |
| 1730 | 0.000360 | 103.286308 |
| 1750 | 0.000369 | 104.886496 |
| 1772 | 0.000420 | 106.307723 |
| 1797 | 0.000364 | 107.644075 |


In [ ]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO E SEGMENTAÇÃO (APENAS DRIVE END)
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---
caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários de mapeamento
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}

# --- PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3
passo = int(tamanho_segmento * (1 - sobreposicao_percentual))

# --- 2. CARREGAMENTO E PROCESSAMENTO DOS ARQUIVOS ---
dicionario_de_dataframes = {}
print(f"Iniciando a leitura e segmentação dos arquivos (APENAS DRIVE END) em '{caminho_raiz}'...")

for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)
            
            # Decodificação de metadados
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm = partes[0]
            
            if 'Normal' in nome_arquivo:
                tipo_falha_cod, diametro_falha = 'Normal', 'N/A'
            else:
                tipo_falha_cod = partes[1].split('@')[0]
                diametro_falha = mapa_diametro_falha.get(partes[2], 'Desconhecido')
            
            tipo_falha = mapa_tipo_falha.get(tipo_falha_cod, 'Desconhecido')
            
            # --- AJUSTE: Processar apenas o sensor 'DE' ---
            sensor_cod = 'DE'
            try:
                dados_npz = np.load(caminho_completo)
                if sensor_cod in dados_npz.files:
                    sinal_completo = dados_npz[sensor_cod].ravel()
                    
                    if 'Normal' in nome_arquivo:
                        chave_base = f"{nome_sem_ext}_{sensor_cod}"
                    else:
                        partes_chave = nome_sem_ext.split('_')
                        partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                        chave_base = "_".join(partes_chave)
                        
                    # Loop de segmentação
                    for i, inicio in enumerate(range(0, len(sinal_completo) - tamanho_segmento + 1, passo)):
                        segmento = sinal_completo[inicio : inicio + tamanho_segmento]
                        df_segmento = pd.DataFrame({'amplitude': segmento})
                        
                        # Adiciona metadados
                        df_segmento['arquivo_origem'] = nome_arquivo
                        df_segmento['rotacao_rpm'] = int(rpm)
                        df_segmento['tipo_falha'] = tipo_falha
                        df_segmento['diametro_falha'] = diametro_falha
                        df_segmento['local_sensor'] = 'Drive End'
                        
                        chave_segmento = f"{chave_base}_seg_{i}"
                        dicionario_de_dataframes[chave_segmento] = df_segmento

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

if not dicionario_de_dataframes:
    print(f"\nERRO: Nenhum segmento do Drive End foi carregado.")
else:
    print(f"\nProcesso concluído! {len(dicionario_de_dataframes)} SEGMENTOS do Drive End foram carregados.")
    chave_exemplo = list(dicionario_de_dataframes.keys())[0]
    print(f"\nExemplo de um segmento (chave: '{chave_exemplo}'):")
    print(dicionario_de_dataframes[chave_exemplo].head())
    print(len(dicionario_de_dataframes))

Iniciando a leitura e segmentação dos arquivos (APENAS DRIVE END) em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...

Processo concluído! 3329 SEGMENTOS do Drive End foram carregados.

Exemplo de um segmento (chave: '1730_B_14_DE_seg_0'):
   amplitude      arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.204633  1730_B_14_DE12.npz         1730     Esfera         0.014"   
1   0.248395  1730_B_14_DE12.npz         1730     Esfera         0.014"   
2   0.042324  1730_B_14_DE12.npz         1730     Esfera         0.014"   
3  -0.015615  1730_B_14_DE12.npz         1730     Esfera         0.014"   
4  -0.058555  1730_B_14_DE12.npz         1730     Esfera         0.014"   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  
3329


In [28]:
# =============================================================================
# BLOCO FINAL: GERAÇÃO DE DADOS SINTÉTICOS (APENAS DRIVE END)
# =============================================================================

# --- 1. FUNÇÃO PARA CÁLCULO DE FREQUÊNCIA TEÓRICA ---
def calcular_frequencias_rolamento(n, fr, d, D, phi_graus=0.0):
    """Calcula as frequências teóricas de falha de um rolamento."""
    phi_rad = np.deg2rad(phi_graus)
    termo_comum = (d / D) * np.cos(phi_rad)
    return {
        'Pista Externa': (n * fr / 2) * (1 - termo_comum),
        'Pista Interna': (n * fr / 2) * (1 + termo_comum),
        'Esfera': (D * fr / (2 * d)) * (1 - termo_comum**2)
    }

# --- 2. PARÂMETROS DE GERAÇÃO (APENAS DRIVE END) ---
amplitudes_referencia = {
    'Drive End': {'Esfera': 0.0001, 'Pista Interna': 0.001, 'Pista Externa': 0.0001}
}
multiplicadores = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 5.0, 10.0, 25.0, 50.0, 100.0]
fases_para_adicionar_rad = [
    0, np.pi/4, np.pi/2, 3*np.pi/4,
    np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4
]

# --- 3. IDENTIFICAÇÃO DOS SEGMENTOS NORMAIS ---
segmentos_normais = {
    chave: df for chave, df in dicionario_de_dataframes.items()
    if df['tipo_falha'].iloc[0] == 'Normal' and df['local_sensor'].iloc[0] == 'Drive End'
}
print(f"Encontrados {len(segmentos_normais)} segmentos normais do Drive End para usar como base.")

# --- 4. GERAÇÃO E COMBINAÇÃO DOS SINAIS ---
lista_dados_finais = []
for chave_normal, df_normal in segmentos_normais.items():
    sinal_normal_base = df_normal['amplitude'].values
    rpm_atual = df_normal['rotacao_rpm'].iloc[0]
    
    # Parâmetros do segmento
    N_PONTOS = len(sinal_normal_base)
    duracao_s = N_PONTOS / TAXA_AMOSTRAL
    t = np.linspace(0.0, duracao_s, N_PONTOS, endpoint=False)
    
    # Frequências teóricas
    fr_hz = rpm_atual / 60
    freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_drive_end)
    
    for tipo_falha_sintetica in ['Pista Externa', 'Pista Interna', 'Esfera']:
        freq_teorica = freqs_teoricas[tipo_falha_sintetica]
        amp_ref = amplitudes_referencia['Drive End'][tipo_falha_sintetica]
            
        for mult in multiplicadores:
            for fase in fases_para_adicionar_rad:
                amplitude_final = amp_ref * mult
                sinal_falha_sintetico = amplitude_final * np.sin(2 * np.pi * freq_teorica * t + fase)
                sinal_final_combinado = sinal_normal_base + sinal_falha_sintetico
                
                lista_dados_finais.append({
                    'sinal_final': sinal_final_combinado,
                    'tipo_falha_adicionada': tipo_falha_sintetica,
                    'rpm': rpm_atual,
                    'multiplicador_amplitude': mult,
                    'fase_adicionada_rad': fase,
                    'base_normal': chave_normal,
                    'local_sensor': 'Drive End'
                })

# --- 5. DATAFRAME FINAL COM DADOS SINTÉTICOS ---
df_final_de = pd.DataFrame(lista_dados_finais)

print("\n\n--- Geração de Dados Sintéticos Concluída! ---")
print(f"Total de {len(df_final_de)} segmentos sintéticos gerados para o Drive End.")
print("\n--- Exemplo do DataFrame Final Gerado ---")
print(df_final_de.drop(columns=['sinal_final']).head())

Encontrados 588 segmentos normais do Drive End para usar como base.


--- Geração de Dados Sintéticos Concluída! ---
Total de 169344 segmentos sintéticos gerados para o Drive End.

--- Exemplo do DataFrame Final Gerado ---
  tipo_falha_adicionada   rpm  multiplicador_amplitude  fase_adicionada_rad  \
0         Pista Externa  1730                      0.1             0.000000   
1         Pista Externa  1730                      0.1             0.785398   
2         Pista Externa  1730                      0.1             1.570796   
3         Pista Externa  1730                      0.1             2.356194   
4         Pista Externa  1730                      0.1             3.141593   

            base_normal local_sensor  
0  1730_Normal_FE_seg_0    Drive End  
1  1730_Normal_FE_seg_0    Drive End  
2  1730_Normal_FE_seg_0    Drive End  
3  1730_Normal_FE_seg_0    Drive End  
4  1730_Normal_FE_seg_0    Drive End  
